In [1]:
#to get the files from google drive folder
!pip install kora -q
from kora import drive
drive.link_nbs()

     |████████████████████████████████| 57 kB 1.7 MB/s 
     |████████████████████████████████| 56 kB 2.1 MB/s 
Mounted at /content/drive


In [2]:
import pandas as pd
import requests
import io
import numpy as np  
from datetime import date, timedelta
import re
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
#from github import Github
#import github
import torch
import torch.nn as nn
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader
# Import nn.functional
import torch.nn.functional as F
import torch.optim as optim
from typing import Union, Tuple
import os
import sys
import time
from collections import OrderedDict
from sklearn.preprocessing import MinMaxScaler
from statistics import mean
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
import math
import random
import imageio
from data_preparation import create_dataset, data_preparation
from model_fit_code import fit
#from sklearn.metrics import mean_absolute_percentage_error
matplotlib.style.use('seaborn')
%matplotlib inline


In [3]:
# generalised implementation of multilayer perceptron
class MLP(nn.Module):
  def __init__(self, input_dim, layers,output_dim):
    super(MLP, self).__init__()
    self.input_dim=input_dim #input dimension
    self.n_layers=layers #number of hidden layers
    self.output_dim=output_dim #output layers
    in_features=input_dim
    out_features=16
    layers=[]
    # generalised implementation of layers
    for l in range(self.n_layers): #iteration number of layers
        if l==(self.n_layers-1):
          layers.append(nn.Linear(in_features=in_features, out_features=self.output_dim)) #fully connected nn layer
        else:
          layers.append(nn.Linear(in_features=in_features, out_features=out_features)) #fully coonected nn layer
        if l%2==0:
          layers.append(nn.Tanh()) #tanh activation function
        else:
          layers.append(nn.ELU(inplace=True)) #elu activation function
        in_features=out_features
        out_features=int(out_features/2)
    self.body = nn.Sequential(*layers) # body of architecture
  def forward(self, x):
    b, n_steps, features = x.shape
    #print(b,n_steps, features)
    x = x.reshape([b,n_steps*features]) #reshaping 
    return self.body(x)

In [5]:
Shortlisted_States=['Karnataka','Maharashtra','Uttar-Pradesh','Kerala','Tamil-Nadu']
results_mlp=[]
for state in Shortlisted_States: #state iteration
  best_models=[]
  # getting data from github
  df=pd.read_csv("https://raw.githubusercontent.com/sureshkuc/Data-Science-in-Life-Science-Project/main/Indian-States-Covid19-Datasets/"+state+".csv", parse_dates=["Date"]).drop(columns =["Unnamed: 0"])
  df = df[df["Date"] > "2020-03-10"] # data March 10, 2020 onwards
  df = df.set_index("Date")
  df = df[['Confirmed', 'Recovered', 'Deceased', 'New_Confirmerd', 'New_Deaths', 'New_Recovered']] # selected features

  time_step=[5,7,15,30] # time step
  Number_of_feature=[1,2,3,4,5,6] # list of feature index
  multi_feature=True
  for n_f in Number_of_feature: # feature iteration 
    for t_s in time_step: #time step iteration 
      #data preprocessing
      train_loader, test_loader,scalar = data_preparation(df, scaling_range=(0,1),time_step=t_s,number_feature=n_f, response_variable_index=0,data_split_ratio=0.8)
      for n_layers in range(1,3,1): # layers iteration
          
          max_epochs=100 # max epoch 
          # setting seed to reproduce the results
          random.seed(42)
          torch.manual_seed(42)
          np.random.seed(42)
          #CNN model with L1 loss
          #best_model=Call_CNN_model(state,dataset=(train_loader, test_loader), lr=1e-2,criterion=nn.L1Loss(),max_epochs=max_epochs)
          fc_model = MLP(input_dim=n_f*t_s, layers=n_layers,output_dim=1) #mlp model initialization 
          cuda=torch.cuda.is_available() #checking cuda is available or not
          if cuda:
            fc_model = fc_model.cuda()
          fc_optim = optim.SGD(fc_model.parameters(), lr=1e-2, momentum=0.9) #sgd optimizer
          #fc_optim = optim.Adam(fc_model.parameters(), lr=1e-3)
          train_losses,test_losses,best_model = fit(fc_model, fc_optim,nn.L1Loss(),(train_loader, test_loader), max_epochs=max_epochs,cuda=cuda)
          #print(f'\nTraining took {end-start}s!')
          #plot_loss(max_epochs,train_losses,test_losses,model_name='CNN for '+state)
          fc_model = MLP(input_dim=n_f*t_s, layers=n_layers,output_dim=1) 
          fc_model.load_state_dict(best_model) #to load the best model
          fc_model.eval() #setting model evaluation
          test_x,test_y=test_loader
          predictions=fc_model(test_x)
          test_y=test_y.cpu().detach().numpy()
          predictions=predictions.cpu().detach().numpy()
          mae=mean_absolute_error(test_y,predictions)
          rmse=math.sqrt(mean_squared_error(test_y,predictions))
          #mape=mean_absolute_percentage_error(test_y,predictions)
          r2s=r2_score(test_y,predictions)
          results_mlp.append([state,n_f,t_s,n_layers,mae,rmse,r2s])
          print(state,'n_f',n_f,'t_s',t_s,'n_layers',n_layers,'Error',mae,rmse,r2s)

(384, 5, 1)
(384, 5, 1) (384, 1)
Epoch: 0/10  Loss: 0.023928 Test loss: 0.022528Karnataka n_f 1 t_s 5 n_layers 1 Error 0.11827387 0.14151160572658333 0.6406263870897307
Epoch: 0/10  Loss: 0.134110 Test loss: 0.131655Karnataka n_f 1 t_s 5 n_layers 2 Error 0.066243224 0.07682007613291669 0.894096196490626
(384, 7, 1)
(384, 7, 1) (384, 1)
Epoch: 0/10  Loss: 0.060060 Test loss: 0.026694Karnataka n_f 1 t_s 7 n_layers 1 Error 0.14445467 0.16233909944476438 0.5095767379266047
Epoch: 0/10  Loss: 0.074036 Test loss: 0.001016Karnataka n_f 1 t_s 7 n_layers 2 Error 0.09359478 0.1015432091879048 0.8081213851448048
(384, 15, 1)
(384, 15, 1) (384, 1)
Epoch: 0/10  Loss: 0.014661 Test loss: 0.033200Karnataka n_f 1 t_s 15 n_layers 1 Error 0.1596317 0.17480917876883198 0.3061190615398752
Epoch: 0/10  Loss: 0.068652 Test loss: 0.004385Karnataka n_f 1 t_s 15 n_layers 2 Error 0.19264172 0.20854234740736527 0.012482087654758556
(384, 30, 1)
(384, 30, 1) (384, 1)
Epoch: 0/10  Loss: 0.048854 Test loss: 0.07537